In [2]:
import pandas as pd
import numpy as np
import torch
import os
dataset_path = "/Users/rdubey/mysrc/random-stuff/esci-data/shopping_queries_dataset"

""" 0. Init variables """
col_query = "query"
col_query_id = "query_id"
col_product_id = "product_id" 
col_product_title = "product_title"
col_product_locale = "product_locale"
col_small_version = "small_version"
col_split = "split"
col_scores = "scores"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
locale = 'us'
""" 1. Load data """    
df_examples = pd.read_parquet(os.path.join(dataset_path, 'shopping_queries_dataset_examples.parquet'))
df_products = pd.read_parquet(os.path.join(dataset_path, 'shopping_queries_dataset_products.parquet'))

df_examples_products = pd.merge(
    df_examples,
    df_products,
    how='left',
    left_on=[col_product_locale, col_product_id],
    right_on=[col_product_locale, col_product_id]
)
df_examples_products = df_examples_products[df_examples_products[col_small_version] == 1]
df_examples_products = df_examples_products[df_examples_products[col_split] == "test"]
df_examples_products = df_examples_products[df_examples_products[col_product_locale] == locale]

features_query = df_examples_products[col_query].to_list()
features_product = df_examples_products[col_product_title].to_list()
n_examples = len(features_query)
scores = np.zeros(n_examples)

In [3]:
df_products.columns

Index(['product_id', 'product_title', 'product_description',
       'product_bullet_point', 'product_brand', 'product_color',
       'product_locale'],
      dtype='object')

In [4]:
pd.set_option('display.max_colwidth', None)

In [5]:
# df_examples_products[['query_id', 'query', 'product_title','esci_label', 'product_bullet_point', 'product_description']]

In [6]:
df_examples_products.head()

,example_id,query,query_id,product_id,product_locale,esci_label,small_version,large_version,split,product_title,product_description,product_bullet_point,product_brand,product_color
32,32,!qscreen fence without holes,2,B07D7TBSGH,us,I,1,1,test,"FOTMISHU 6Pcs Greenhouse Hoops Rust-Free Grow Tunnel Tunnel, 4ft Long Steel with Plastic Coated Plant Supports for Garden Fabric, Plant Support Garden Stakes","<p><b>Description:</b><br><b>Material:</b>plastic coated steel pipe<b>Size</b>Length 120 (the size before bending), width 50 * height 48CM(The size after bending)<b>Weight:</b> 720g<b>Color:</b>green<b>Packing includ:</b>6pcs hoops/bag<b>Product Usage:</b><br>1, can be used for the traction of Tomatoes vine, vines and some natural climbing plants. with support<br>2, can be used as a support for the greenhouse<br>3, Garden Hoops provide convenience for your garden plant work<br><b>Features:</b><br>1, unique and handsome in appearance<br>2, good material, strong antioxidant capacity<br>3, green and health.<br><b>Installation steps:</b><br>►Gathering all materials<br>►Bend the hoops<br>►Outline Your Low Tunnel Greenhouse<br>►Put Your Hoops in Place<br>►Tie a Ridgeline<br>►Time to Place the cover<br>►Tie Down the Ends with pins<br>►The distination between 2 hoops should be less than 23.6 inches","►Material -- High-quality plastic coated steel pipe, not easy to rust, can be used for a long time.\n►Size -- Curved tube long 120cm/4ft (the size before bending), wide 50cm/19.7inch, high 48cm/18.9inch (the size after bending), Tube diameter 11mm/0.43inch. Each package contains 6 Hoops.Garden Fabric Not Included.\n►Easy to set up -- Comes complete With sharpened points on both sides for ease of placement. Washable, Reusable, Flat for storage. Creates a favorable microclimate for your plants.\n►Protecting Plants -- Use hoops to support garden row covers, protecting plants from frost, insects, birds, or intense sun. Can be used with insect-proof nets, shading nets, insulation film and so on.\n►Features -- Design for plant cover, frost protection cover and mini greenhouse,provides ideal environment for rapid germination and growth of young plants",FOTMISHU,green
33,33,!qscreen fence without holes,2,B07DHT2WZK,us,I,1,1,test,"Zippity Outdoor Products ZP19028 Unassembled Madison Vinyl Gate Kit with Fence Wings, White",None,"Designed for the Madison Vinyl Fence (item ZP19001)\nTo install, dig holes and secure the extended posts into the ground; use gravel or concrete for extra stability (optional)\nKit includes small stainless latch (1), hinges (2), supporting posts (2), decorative fence wings with posts (2) and post caps (6)\nThe entire gate kit with wings spans 61-3/8in. Gate without wings is 36-7/8in. Gate must be installed with wings for optimal stability\nAssembly and installation required",Zippity Outdoor Products,White
34,34,!qscreen fence without holes,2,B07DHX8YH2,us,E,1,1,test,"Zippity Outdoor Products ZP19026 Lightweight Portable Vinyl Picket Fence Kit w/Metal Base(42"" H x 92"" W), White",None,Designed as a portable fence for temporary events\nRust-free metal bases are included; bases have small holes if you need to screw the base into wood or concrete (fence will stand on its own without screws)\nThis fence should not be used for serious crowd control or child or pet safety; it is designed to be used as a psychological or aesthetic barrier and not a physical one\nEach panel includes a connector piece to attach to an adjacent panel in a straight line or at a right angle; this makes this fence fully scalable to add as many panels as you need for your event\nAssembly required. Top Shape-Flat,Zippity Outdoor Products,White
35,35,!qscreen fence without holes,2,B07DS1YCRZ,us,S,1,1,test,ColourTree 4' x 50' Green Fence Privacy Screen Windscreen Cover Fabric Shade Tarp Netting Mesh Cloth - Commercial Grade 170 GSM - Cable Zip Ties Included - We Make Custom Size,None,"★【Virgin, UV Stabilized Compounds & Thick Material】: The only fence screen pro

In [7]:
df_examples_products.isna().any()

example_id              False
query                   False
query_id                False
product_id              False
product_locale          False
esci_label              False
small_version           False
large_version           False
split                   False
product_title           False
product_description      True
product_bullet_point     True
product_brand            True
product_color            True
dtype: bool

In [8]:
df_examples_products['product_description'].fillna(" ", inplace=True)
df_examples_products['product_bullet_point'].fillna(" ", inplace=True)
df_examples_products['product_brand'].fillna(" ", inplace=True)
df_examples_products['product_color'].fillna(" ", inplace=True)

In [9]:
import re
def clean_text_column(x):
    x = x.lower() # lowercase everything
    x = ' '.join(x.strip().splitlines())
    x = x.encode('ascii', 'ignore').decode()  # remove unicode characters
    x = re.sub(r'https*\S+', ' ', x) # remove links
    x = re.sub(r'http*\S+', ' ', x)

    # cleaning up text
    x = re.sub(r'\'\w+', '', x) 
    
    x = re.sub(r'\w*\d+\w*', '', x)
    
    x = re.sub(r'\s{2,}', ' ', x)
    
    x = re.sub(r'\s[^\w\s]\s', '', x)
#     print(x)
    x = remove_html_tags(x)
    return x

def remove_html_tags(text):
    """Remove html tags from a string"""
    import re
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)

In [10]:
s = u"""<p><b>Description:</b><br><b>Material:</b>plastic coated steel pipe<b>Size</b>Length 120 (the size before bending), width 50 * height 48CM(The size after bending)<b>Weight:</b> 720g<b>Color:</b>green<b>Packing includ:</b>6pcs hoops/bag<b>Product Usage:</b><br>1, can be used for the traction of Tomatoes vine, vines and some natural climbing plants. with support<br>2, can be used as a support for the greenhouse<br>3, Garden Hoops provide convenience for your garden plant work<br><b>Features:</b><br>1, unique and handsome in appearance<br>2, good material, strong antioxidant capacity<br>3, green and health.<br><b>Installation steps:</b><br>►Gathering all materials<br>►Bend the hoops<br>►Outline Your Low Tunnel Greenhouse<br>►Put Your Hoops in Place<br>►Tie a Ridgeline<br>►Time to Place the cover<br>►Tie Down the Ends with pins<br>►The distination between 2 hoops should be less than 23.6 inches"""
clean_text_column(s)

'description:material:plastic coated steel pipesizelength (the size before bending), widthheight (the size after bending)weight: color:greenpacking includ: hoops/bagproduct usage:, can be used for the traction of tomatoes vine, vines and some natural climbing plants. with support, can be used as a support for the greenhouse, garden hoops provide convenience for your garden plant workfeatures:, unique and handsome in appearance, good material, strong antioxidant capacity, green and health.installation steps:gathering all materialsbend the hoopsoutline your low tunnel greenhouseput your hoops in placetie a ridgelinetime to place the covertie down the ends with pinsthe distination between hoops should be less thaninches'

In [11]:
df_examples_products_clean = df_examples_products.copy()

In [12]:
df_examples_products_clean['query'] = df_examples_products_clean['query'].apply(clean_text_column) 
df_examples_products_clean['product_title'] = df_examples_products_clean['product_title'].apply(clean_text_column) 
df_examples_products_clean['product_description'] = df_examples_products_clean['product_description'].apply(clean_text_column) 
df_examples_products_clean['product_bullet_point'] = df_examples_products_clean['product_bullet_point'].apply(clean_text_column) 
df_examples_products_clean[['query', 'product_title', 'product_bullet_point', 'product_description']].head(50)

,query,product_title,product_bullet_point,product_description
32,!qscreen fence without holes,"fotmishu greenhouse hoops rust-free grow tunnel tunnel, long steel with plastic coated plant supports for garden fabric, plant support garden stakes","material--high-qualityplasticcoatedsteelpipe,noteasytorust,canbeusedforalongtime. size--/(thesizebeforebending),/.,/.(thesizeafterbending),/...gardenfabricnotincluded. easytosetup--comescompletewithsharpenedpointsonbothsidesforeaseofplacement.washable,reusable,flatforstorage.createsafavorablemicroclimateforyourplants. protectingplants--usehoopstosupportgardenrowcovers,protectingplantsfromfrost,insects,birds,orintensesun.canbeusedwithinsect-proofnets,shadingnets,insulationfilmandsoon. features--designforplantcover,frostprotectioncoverandminigreenhouse,providesidealenvironmentforrapidgerminationandgrowthofyoungplants","description:material:plasticcoatedsteelpipesize(thesizebeforebending),*(thesizeafterbending)weight:color:greenpackinginclud:/bagproductusage:,canbeusedforthetractionoftomatoesvine,vinesandsomenaturalclimbingplants.withsupport,canbeusedasasupportforthegreenhouse,gardenhoopsprovideconvenienceforyourgardenplantworkfeatures:,uniqueandhandsomeinappearance,goodmaterial,strongantioxidantcapacity,greenandhealth.installationsteps:gatheringallmaterialsbendthehoopsoutlineyourlowtunnelgreenhouseputyourhoopsinplacetiearidgelinetimetoplacethecovertiedowntheendswithpins."
33,!qscreen fence without holes,"zippity outdoor products unassembled madison vinyl gate kit with fence wings, white","designed for the madison vinyl fence (itemto install, dig holes and secure the extended posts into the ground; use gravel or concrete for extra stability (optional) kit includes small stainless latch (), hinges (), supporting posts (), decorative fence wings with posts () and post caps () the entire gate kit with wings spans -/. gate without wings is -/. gate must be installed with wings for optimal stability assembly and installation required",
34,!qscreen fence without holes,"zippity outdoor products lightweight portable vinyl picket fence kit w/metal base("" h xw), white",designed as a portable fence for temporary events rust-free metal bases are included; bases have small holes if you need to screw the base into wood or concrete (fence will stand on its own without screws) this fence should not be used for serious crowd control or child or pet safety; it is designed to be used as a psychological or aesthetic barrier and not a physical one each panel includes a connector piece to attach to an adjacent panel in a straight line or at a right angle; this makes this fence fully scalable to add as many panels as you need for your event assembly required. top shape-flat,
35,!qscreen fence without holes,colourtreexgreen fence privacy screen windscreen cover fabric shade tarp netting mesh clothcommercial grade gsmcable zip ties includedwe make custom size,"virgin, uv stabilized compoundsthick material: the only fence screen product on the market that are made of gsmvirgin (non-recycled) hdpe material. our virgin hdpe fabric went through hours of laboratory uv testing and it is proved to bemore durable compared to recycled hdpe under sun exposure, fabric has been treated with uv stabilized compound, preventing fence screen from dried out, tearing and fading, significantly extend the lifespan. privacy solutionvisibility blockage: most economical way for privacy over chain link fence, porous decorative concealment, existing cyclone wire fence, wood retaining wall, welded wire fence, chain link fence, privacy barrier around swimming pool, block nose neighbor see through, add privacy to acre back yard backdrop, front yard, patio, garden and porch; hide street, road, construction site, pet kennel pen, block surrounding noise and street view, stops dogs from barking at neighb comes with the extra stainless grommets and reinforced binding: grommets are made of stainless copper, each corner has grommets (compares to 

In [13]:
df_examples_products_clean.head(50)

,example_id,query,query_id,product_id,product_locale,esci_label,small_version,large_version,split,product_title,product_description,product_bullet_point,product_brand,product_color
32,32,!qscreen fence without holes,2,B07D7TBSGH,us,I,1,1,test,"fotmishu greenhouse hoops rust-free grow tunnel tunnel, long steel with plastic coated plant supports for garden fabric, plant support garden stakes","description:material:plasticcoatedsteelpipesize(thesizebeforebending),*(thesizeafterbending)weight:color:greenpackinginclud:/bagproductusage:,canbeusedforthetractionoftomatoesvine,vinesandsomenaturalclimbingplants.withsupport,canbeusedasasupportforthegreenhouse,gardenhoopsprovideconvenienceforyourgardenplantworkfeatures:,uniqueandhandsomeinappearance,goodmaterial,strongantioxidantcapacity,greenandhealth.installationsteps:gatheringallmaterialsbendthehoopsoutlineyourlowtunnelgreenhouseputyourhoopsinplacetiearidgelinetimetoplacethecovertiedowntheendswithpins.","material--high-qualityplasticcoatedsteelpipe,noteasytorust,canbeusedforalongtime. size--/(thesizebeforebending),/.,/.(thesizeafterbending),/...gardenfabricnotincluded. easytosetup--comescompletewithsharpenedpointsonbothsidesforeaseofplacement.washable,reusable,flatforstorage.createsafavorablemicroclimateforyourplants. protectingplants--usehoopstosupportgardenrowcovers,protectingplantsfromfrost,insects,birds,orintensesun.canbeusedwithinsect-proofnets,shadingnets,insulationfilmandsoon. features--designforplantcover,frostprotectioncoverandminigreenhouse,providesidealenvironmentforrapidgerminationandgrowthofyoungplants",FOTMISHU,green
33,33,!qscreen fence without holes,2,B07DHT2WZK,us,I,1,1,test,"zippity outdoor products unassembled madison vinyl gate kit with fence wings, white",,"designed for the madison vinyl fence (itemto install, dig holes and secure the extended posts into the ground; use gravel or concrete for extra stability (optional) kit includes small stainless latch (), hinges (), supporting posts (), decorative fence wings with posts () and post caps () the entire gate kit with wings spans -/. gate without wings is -/. gate must be installed with wings for optimal stability assembly and installation required",Zippity Outdoor Products,White
34,34,!qscreen fence without holes,2,B07DHX8YH2,us,E,1,1,test,"zippity outdoor products lightweight portable vinyl picket fence kit w/metal base("" h xw), white",,designed as a portable fence for temporary events rust-free metal bases are included; bases have small holes if you need to screw the base into wood or concrete (fence will stand on its own without screws) this fence should not be used for serious crowd control or child or pet safety; it is designed to be used as a psychological or aesthetic barrier and not a physical one each panel includes a connector piece to attach to an adjacent panel in a straight line or at a right angle; this makes this fence fully scalable to add as many panels as you need for your event assembly required. top shape-flat,Zippity Outdoor Products,White
35,35,!qscreen fence without holes,2,B07DS1YCRZ,us,S,1,1,test,colourtreexgreen fence privacy screen windscreen cover fabric shade tarp netting mesh clothcommercial grade gsmcable zip ties includedwe make custom size,,"virgin, uv stabilized compoundsthick material: the only fence screen product on the market that are made of gsmvirgin (non-recycled) hdpe material. our virgin hdpe fabric went through hours of laboratory uv testing and it is proved to bemore durable compared to recycled hdpe under sun exposure, fabric has been treated with uv stabilized compound, preventing fence screen from dried out, tearing and fading, significantly extend the lifespan. privacy solutionvisibility blockage: most economical way for privacy over chain link fence, porous decorative concealment, existing cyclone wire fence, wood retaining wall, welded wire fence, chain link fence, privacy barrier around swimming pool, block nose neighbor see through, add privacy to acre back yard 

In [14]:
df_examples_products_clean['product_text'] = (
    df_examples_products_clean['product_title'] + '. ' + df_examples_products_clean['product_description'] + 
    '. ' + df_examples_products_clean['product_bullet_point']
)
df_examples_products_clean[['query', 'esci_label', 'product_text']].head()

,query,esci_label,product_text
32,!qscreen fence without holes,I,"fotmishu greenhouse hoops rust-free grow tunnel tunnel, long steel with plastic coated plant supports for garden fabric, plant support garden stakes. description:material:plasticcoatedsteelpipesize(thesizebeforebending),*(thesizeafterbending)weight:color:greenpackinginclud:/bagproductusage:,canbeusedforthetractionoftomatoesvine,vinesandsomenaturalclimbingplants.withsupport,canbeusedasasupportforthegreenhouse,gardenhoopsprovideconvenienceforyourgardenplantworkfeatures:,uniqueandhandsomeinappearance,goodmaterial,strongantioxidantcapacity,greenandhealth.installationsteps:gatheringallmaterialsbendthehoopsoutlineyourlowtunnelgreenhouseputyourhoopsinplacetiearidgelinetimetoplacethecovertiedowntheendswithpins.. material--high-qualityplasticcoatedsteelpipe,noteasytorust,canbeusedforalongtime. size--/(thesizebeforebending),/.,/.(thesizeafterbending),/...gardenfabricnotincluded. easytosetup--comescompletewithsharpenedpointsonbothsidesforeaseofplacement.washable,reusable,flatforstorage.createsafavorablemicroclimateforyourplants. protectingplants--usehoopstosupportgardenrowcovers,protectingplantsfromfrost,insects,birds,orintensesun.canbeusedwithinsect-proofnets,shadingnets,insulationfilmandsoon. features--designforplantcover,frostprotectioncoverandminigreenhouse,providesidealenvironmentforrapidgerminationandgrowthofyoungplants"
33,!qscreen fence without holes,I,"zippity outdoor products unassembled madison vinyl gate kit with fence wings, white. . designed for the madison vinyl fence (itemto install, dig holes and secure the extended posts into the ground; use gravel or concrete for extra stability (optional) kit includes small stainless latch (), hinges (), supporting posts (), decorative fence wings with posts () and post caps () the entire gate kit with wings spans -/. gate without wings is -/. gate must be installed with wings for optimal stability assembly and installation required"
34,!qscreen fence without holes,E,"zippity outdoor products lightweight portable vinyl picket fence kit w/metal base("" h xw), white. . designed as a portable fence for temporary events rust-free metal bases are included; bases have small holes if you need to screw the base into wood or concrete (fence will stand on its own without screws) this fence should not be used for serious crowd control or child or pet safety; it is designed to be used as a psychological or aesthetic barrier and not a physical one each panel includes a connector piece to attach to an adjacent panel in a straight line or at a right angle; this makes this fence fully scalable to add as many panels as you need for your event assembly required. top shape-flat"
35,!qscreen fence without holes,S,"colourtreexgreen fence privacy screen windscreen cover fabric shade tarp netting mesh clothcommercial grade gsmcable zip ties includedwe make custom size. . virgin, uv stabilized compoundsthick material: the only fence screen product on the market that are made of gsmvirgin (non-recycled) hdpe material. our virgin hdpe fabric went through hours of laboratory uv testing and it is proved to bemore durable compared to recycled hdpe under sun exposure, fabric has been treated with uv stabilized compound, preventing fence screen from dried out, tearing and fading, significantly extend the lifespan. privacy solutionvisibility blockage: most economical way for privacy over chain link fence, porous decorative concealment, existing cyclone wire fence, wood retaining wall, welded wire fence, chain link fence, privacy barrier around swimming pool, block nose neighbor see through, add privacy to acre back yard backdrop, front yard, patio, garden and porch; hide street, road, construction site, pet kennel pen, block surrounding noise and street view, stops dogs from barking at neighb comes with the extra stainless grommets and reinforced binding: grommets are made of stainless copper, each corner has grommets (compares to similar products on 

In [16]:
from transformers import AutoTokenizer
from sentence_transformers import SentenceTransformer
model_name = 'all-distilroberta-v1'
model = SentenceTransformer(model_name)
model.encode("hello world")

array([ 3.96649987e-02, -2.92933378e-02, -2.77088415e-02, -5.49586862e-02,
        3.40879373e-02, -4.12879288e-02,  1.73210036e-02, -4.14500479e-03,
        2.39177924e-02, -8.85709771e-04,  7.50699267e-03,  1.11439647e-02,
       -2.54435372e-03, -4.10563536e-02, -5.81563078e-02, -9.57910717e-03,
        5.94797544e-03, -5.65076061e-02, -8.11948553e-02, -3.86233046e-03,
        3.56025776e-05,  2.23608203e-02, -1.16068590e-02,  2.46384796e-02,
        7.88948685e-02,  6.59193546e-02, -8.31936859e-03, -1.72879491e-02,
        1.79381501e-02,  1.59524791e-02,  3.87127488e-03, -3.54216471e-02,
       -5.05456775e-02, -9.94721893e-03, -2.75229849e-02, -4.95090731e-04,
        3.46693234e-03, -2.96155065e-02,  6.80901855e-02,  2.85488274e-02,
        1.95989665e-02, -3.92260887e-02,  4.25860807e-02, -6.37953868e-03,
       -2.39171181e-02, -7.62367668e-03, -3.41702998e-02, -6.29468560e-02,
       -1.60953996e-03,  5.92776462e-02, -2.90842284e-03, -2.07001977e-02,
       -4.96227145e-02,  

In [34]:
import torch.nn.functional as F
# input1 = torch.randn(100, 128)
# input2 = torch.randn(100, 128)
# output = F.cosine_similarity(input1, input2)
# mm = torch.mm(input1, input2.transpose(0, 1))
# print(mm.shape)
# print(output.shape)
# print(output)

In [117]:
# mm

In [118]:
query_ids = df_examples_products_clean['query_id'].tolist()
qids = list(set(query_ids))
print(len(qids))
product_ids = df_examples_products_clean['product_id'].tolist()
print(len(query_ids), len(product_ids))
scores = []
for i, qi in enumerate(qids):
    if i%500==0:
        print(i)
#     print(i, qi)
    temp = df_examples_products_clean[df_examples_products.query_id==qi]
    features_query = temp['query']
    features_product = temp['product_text']
    query_emb = torch.tensor(model.encode(features_query.tolist()))
    prod_emb = torch.tensor(model.encode(features_product.tolist()))
#     print(query_emb.shape, prod_emb.shape)
#     dot_product = torch.diagonal(torch.mm(query_emb, prod_emb.transpose(0, 1)).to('cpu'))
    cos_sim = F.cosine_similarity(query_emb, prod_emb)
#     print(q_scores.shape, q_scores.tolist())
    scores.extend(cos_sim.tolist())
#     break
#     if i==3:
#         break
print(len(scores))

8956
181701 181701
0


[E thread_pool.cpp:113] Exception in thread pool task: mutex lock failed: Invalid argument
[E thread_pool.cpp:113] Exception in thread pool task: mutex lock failed: Invalid argument
[E thread_pool.cpp:113] Exception in thread pool task: mutex lock failed: Invalid argument


KeyboardInterrupt: 

In [ ]:
# result_df = pd.DataFrame({
#     'query_id': query_ids,
#     'product_id': product_ids,
#     'scores': scores
# })
# result_df.head()

In [21]:
df_examples_products_clean.query_id.describe()

count    181701.000000
mean      55613.136378
std       34910.163897
min           2.000000
25%       24668.000000
50%       55219.000000
75%       86083.000000
max      130378.000000
Name: query_id, dtype: float64

In [37]:
%%time
df_sample = df_examples_products_clean[df_examples_products_clean.query_id < 10000]
len(df_sample.query_id.unique())

CPU times: user 13 ms, sys: 34.3 ms, total: 47.3 ms
Wall time: 46.2 ms


996

In [38]:
from tqdm import tqdm

def embed_query_products(i, group_df):
#     i, group_df = grouping
    features_query = group_df['query']
    features_product = group_df['product_text']
    q_ids = [i]*len(group_df)
    p_ids = group_df['product_id']
    query_emb = torch.tensor(model.encode(features_query.tolist()))
    prod_emb = torch.tensor(model.encode(features_product.tolist()))
#     q_scores = torch.diagonal(torch.mm(query_emb, prod_emb.transpose(0, 1)).to('cpu'))
    cos_sim = F.cosine_similarity(query_emb, prod_emb)
    return pd.DataFrame({col_query_id: q_ids, col_product_id: p_ids, col_scores: cos_sim})
# groups = df_examples_products_clean.groupby('query_id')
# triplets = []
# for g in tqdm(groups):
#     triplets.append(embed_query_products(g))

# result_df = pd.concat(triplets)

In [39]:
%%time
from joblib import Parallel, delayed
import multiprocessing
# https://stackoverflow.com/questions/26187759/parallelize-apply-after-pandas-groupby
def applyParallel(dfGrouped, func):
    retLst = Parallel(n_jobs=multiprocessing.cpu_count())(delayed(func)(name, group) for name, group in dfGrouped)
    return pd.concat(retLst)
from multiprocessing import Pool, cpu_count

# def applyParallel(dfGrouped, func):
#     with Pool(cpu_count()) as p:
#         ret_list = p.map(func, [group for name, group in dfGrouped])
#     return pandas.concat(ret_list)
groups = df_sample.groupby('query_id')
result_df = applyParallel(groups, embed_query_products)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

/Users/rdubey/miniconda3/envs/deep-learning/lib/python3.9/site-packages/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


/Users/rdubey/miniconda3/envs/deep-learning/lib/python3.9/site-packages/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


...

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


/Users/rdubey/miniconda3/envs/deep-learning/lib/python3.9/site-packages/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


/Users/rdubey/miniconda3/envs/deep-learning/lib/python3.9/site-packages/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
CPU times: user 47.7 s, sys: 2min 35s, total: 3min 23s
Wall time: 56min 15s


In [40]:
result_df.head()

,query_id,product_id,scores
32,2,B07D7TBSGH,0.227922
33,2,B07DHT2WZK,0.382355
34,2,B07DHX8YH2,0.411532
35,2,B07DS1YCRZ,0.429586
36,2,B07DS3J3MB,0.431270


In [42]:
df_hypothesis = result_df.sort_values(by=[col_query_id, col_scores], ascending=False)
df_hypothesis.head(10)

,query_id,product_id,scores
218871,9990,B01LY7YTGX,0.690298
218884,9990,B0187P09LO,0.630629
218886,9990,B01872PR3C,0.625512
218856,9990,B01M1R695U,0.625484
218870,9990,B07XBPGWH6,0.610600
218873,9990,B01LXFSGH6,0.608016
218861,9990,B01M1H5CLL,0.602678
218875,9990,B0187P07AC,0.600610
218883,9990,B00CE5C41W,0.595815
218860,9990,B01LYJS16F,0.592405


In [43]:
""" 4. Prepare hypothesis file """
HYPOTHESIS_TASK1_PATH="./hypothesis"
HYPOTHESIS_PATH_FILE=f"{HYPOTHESIS_TASK1_PATH}/task_1_ranking_model_baseline_triplet_product_cosine_{locale}.csv"

In [44]:
# df_hypothesis = pd.DataFrame({
#     col_query_id : df_examples_products[col_query_id].to_list(),
#     col_product_id : df_examples_products[col_product_id].to_list(),
#     col_scores : scores,
# })
df_hypothesis[[col_query_id, col_product_id]].to_csv(
    HYPOTHESIS_PATH_FILE,
    index=False,
    sep=',',
)